### 1. 모듈 불러오기

In [1]:
from konlpy.tag import Twitter, Mecab, Kkma
twitter = Twitter(); mecab = Mecab(); kkma = Kkma()

from collections import Counter

### 2. 파일 불러오기

In [2]:
pos_sents = open("positive_sents.txt").readlines()[:3]
neg_sents = open("negative_sents.txt").readlines()[:3]

In [3]:
print(pos_sents)
print(neg_sents)

['영어로 할 수 있는 활동은 거의 다 한다고 보시면 됩니다.\n', '이메일,에세이,프레젠테이션,디스커션,서베이,리포트.\n', '특히 마지막에 기말고사 대체 과제가 정말 빡셉니다.\n']
['네이트평이 너무 안 좋아서 걱정 많이 했는데 듣던 대로 형식주의자인데 하라는 거만 제대로 하면 성적은 무난하게 나옵니다.\n', '영어 실력이랑 큰 관계는 없는 것 같아요.\n', '진짜 하라는 대로만 하면 됨.\n']


### 3. BOW

In [4]:
def bow(data, targets):
    
    output = []
    
    for i in range(0, len(data)):
        words = twitter.pos(data[i], norm=True, stem=True)
        words = [word for (word, tag) in words if word in targets]

        cnt = Counter(words)
        
        targets_output = [0]*len(targets)
        
        for word in cnt.keys():
            if word in targets:
                idx = targets.index(word)
                targets_output[idx] = cnt[word]

        output.append(targets_output)
        
        
    return output

In [5]:
targets = ['영어', '하늘', '하다', '션']
print(bow(pos_sents, targets))

[[1, 0, 2, 0], [0, 0, 0, 2], [0, 0, 0, 0]]


### 4. N-gram

#### 1) cleanInput

In [6]:
def cleanInput(data, tagger, rm_stopwords=True):
    
    output = []
    
    for i in range(0, len(data)):
        
        if tagger == twitter:
            if rm_stopwords:
                sent = twitter.pos(data[i], norm=True, stem=True)
                sent = [word for (word, tag) in sent if tag not in twitter_fms]
            else:
                sent = twitter.pos(data[i])
                sent = [word for (word, tag) in sent]
                
        elif tagger == mecab:
            if rm_stopwords:
                sent = mecab.pos(data[i])
                sent = [word for (word, tag) in sent if tag in mecab_fms]
            else:
                sent = mecab.pos(data[i])
                sent = [word for (word, tag) in sent]
        
        else:
            if rm_stopwords:
                sent = kkma.pos(data[i])
                sent = [word for (word, tag) in sent if tag in kkma_fms]
            else:
                sent = kkma.pos(data[i])
                sent = [word for (word, tag) in sent]
                
        output.append(sent)
                
    return output

In [13]:
print(pos_sents)

['영어로 할 수 있는 활동은 거의 다 한다고 보시면 됩니다.\n', '이메일,에세이,프레젠테이션,디스커션,서베이,리포트.\n', '특히 마지막에 기말고사 대체 과제가 정말 빡셉니다.\n']


In [9]:
twitter_fms = ['Josa', 'Punctuation', 'Suffix']
mecab_fms = ['NNG', 'NNP', 'NR', 'NP', 'VV', 'VA', 'VCP', 'VCN', 'MM', 'MAG', 'XR']
kkma_fms = ['NNG', 'NNP', 'NR', 'NP', 'VV', 'VA', 'VCP', 'VCN', 'MDN', 'MDT', 'MAG', 'XR', 'UN']

print(cleanInput(pos_sents, twitter, True))
print(cleanInput(pos_sents, mecab, True))
print(cleanInput(pos_sents, kkma, True))
print('-')
# print(cleanInput(pos_sents, twitter, False))
# print(cleanInput(pos_sents, mecab, False))
# print(cleanInput(pos_sents, kkma, False))

[['영어', '하다', '수', '있다', '활동', '거의', '다', '하다', '보시', '되다'], ['이메일', '에세이', '프레', '젠', '테이', '션', '디스', '크다', '션', '서다', '베이', '리포트'], ['특히', '마지막', '기말고사', '대체', '과제', '정말', '빡셉니']]
[['영어', '있', '활동', '거의', '다'], ['이메일', '에세이', '프레젠테이션', '디스', '커', '서베이', '리포트'], ['특히', '마지막', '기말고사', '대체', '과제', '정말', '빡', '셉']]
[['영어', '하', '있', '활동', '거의', '다', '하', '보시', '되'], ['이메일', '에세이', '프', '레', '젠', '테', '션', '디스커션', '서베이', '리포트'], ['특히', '마지막', '기말', '고사', '대체', '과제', '정말', '빡세']]
-


#### 2) Ngram

In [10]:
def ngram(data, n, tagger, rm_stopwords=True):
    
    output = []

    if rm_stopwords:
        results = cleanInput(data, tagger, True)
    else:
        results = cleanInput(data, tagger, False)
    
    for i in range(0, len(results)):
        for j in range(0, len(results[i])-(n-1)):
            a = results[i][j:j+n]
            output.append(a)
            
    return output

In [11]:
print(ngram(pos_sents, 2, twitter, False))

[['영어', '로'], ['로', '할'], ['할', '수'], ['수', '있는'], ['있는', '활동'], ['활동', '은'], ['은', '거의'], ['거의', '다'], ['다', '한'], ['한', '다고'], ['다고', '보시'], ['보시', '면'], ['면', '됩'], ['됩', '니다'], ['니다', '.'], ['이메일', ','], [',', '에세이'], ['에세이', ','], [',', '프레'], ['프레', '젠'], ['젠', '테이'], ['테이', '션'], ['션', ','], [',', '디스'], ['디스', '커'], ['커', '션'], ['션', ','], [',', '서'], ['서', '베이'], ['베이', ','], [',', '리포트'], ['리포트', '.'], ['특히', '마지막'], ['마지막', '에'], ['에', '기말고사'], ['기말고사', '대체'], ['대체', '과제'], ['과제', '가'], ['가', '정말'], ['정말', '빡셉니'], ['빡셉니', '다'], ['다', '.']]
